In [1]:
import cv2
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import exposure
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
image_paths = glob.glob(os.path.join("gambar_buah", '*', '*'))
images = []
for img_path in image_paths:
    img = cv2.imread(img_path)
    if img is not None:
        images.append((img_path, img))

In [3]:
def extract_rgb_histograms(images, bins=32):
    data = []
    for img_path, img in images:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        hist_r = cv2.calcHist([img_rgb], [0], None, [bins], [0, 256]).flatten()
        hist_g = cv2.calcHist([img_rgb], [1], None, [bins], [0, 256]).flatten()
        hist_b = cv2.calcHist([img_rgb], [2], None, [bins], [0, 256]).flatten()
        hist = np.concatenate([hist_r, hist_g, hist_b])
        hist = hist / hist.sum()
        kategori = os.path.normpath(img_path).split(os.sep)[-2]
        nama_file = os.path.basename(img_path)
        data.append({'kategori': kategori, 'filename': nama_file, 'histogram': hist})
    return pd.DataFrame(data)

def extract_hog_features(images, pixels_per_cell=(8, 8), cells_per_block=(2, 2), orientations=9):
    data = []
    for img_path, img in images:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        hog_feat = hog(
            gray,
            orientations=orientations,
            pixels_per_cell=pixels_per_cell,
            cells_per_block=cells_per_block,
            block_norm='L2-Hys',
            feature_vector=True
        )
        kategori = os.path.normpath(img_path).split(os.sep)[-2]
        nama_file = os.path.basename(img_path)
        data.append({'kategori': kategori, 'filename': nama_file, 'hog_feature': hog_feat})
    return pd.DataFrame(data)

In [4]:
if images:
    df_hist = extract_rgb_histograms(images)
    df_hog = extract_hog_features(images)

    df_hist_expanded = pd.DataFrame(df_hist['histogram'].to_list(), columns=[f'hist_{c}_{i}' for c in ['r','g','b'] for i in range(32)])
    df_hist_final = pd.concat([df_hist[['kategori','filename']], df_hist_expanded], axis=1)
    df_hist_final.to_csv("fitur_histogram.csv", index=False)
    
    df_hog_expanded = pd.DataFrame(df_hog['hog_feature'].to_list())
    df_hog_final = pd.concat([df_hog[['kategori','filename']], df_hog_expanded], axis=1)
    df_hog_final.to_csv("fitur_hog.csv", index=False)
    print("CSV fitur_histogram.csv dan fitur_hog.csv berhasil dibuat!")
else:
    print("Tidak ada gambar ditemukan di folder gambar_buah.")

CSV fitur_histogram.csv dan fitur_hog.csv berhasil dibuat!


In [5]:
def get_categories(images):
    return sorted(list(set(os.path.normpath(path).split(os.sep)[-2] for path, _ in images)))

def show_sample_by_category(images):
    categories = get_categories(images)
    dropdown = widgets.Dropdown(options=categories, description='Kategori:')
    output = widgets.Output()

    def on_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with output:
                clear_output(wait=True)
                
                for img_path, img in images:
                    kategori = os.path.normpath(img_path).split(os.sep)[-2]
                    nama_file = os.path.basename(img_path)
                    if kategori == change['new']:
                       
                        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        plt.figure(figsize=(10,4))
                        for i, color in enumerate(['r', 'g', 'b']):
                            hist = cv2.calcHist([img_rgb], [i], None, [32], [0,256])
                            plt.plot(hist, color=color)
                        plt.title(f'RGB Histogram - {kategori}/{nama_file}')
                        plt.xlabel('Bin')
                        plt.ylabel('Frequency')
                        plt.show()
                    
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        hog_feat, hog_image = hog(
                            gray, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2),
                            block_norm='L2-Hys', visualize=True, feature_vector=True
                        )
                        hog_image_rescaled = exposure.rescale_intensity(hog_image, in_range=(0, 10))
                        plt.figure(figsize=(8,4))
                        plt.subplot(1,2,1)
                        plt.imshow(gray, cmap='gray')
                        plt.title('Grayscale')
                        plt.axis('off')
                        plt.subplot(1,2,2)
                        plt.imshow(hog_image_rescaled, cmap='gray')
                        plt.title('HOG Visualization')
                        plt.axis('off')
                        plt.show()
                        break
    dropdown.observe(on_change)
    display(dropdown, output)

    with output:
        on_change({'type':'change','name':'value','new':dropdown.value})

if images:
    show_sample_by_category(images)

Dropdown(description='Kategori:', options=('apple', 'avocado', 'mango', 'orange', 'watermelon'), value='apple'…

Output()